# Add HAProxy controller to HPCC TLS with multiple services
Use a dynamic public IP address to create an HTTPS ingress-HAProxy controller on Azure Kubernetes Service.

An ingress controller is a piece of software that provides reverse proxy, configurable traffic routing, and TLS termination for Kubernetes services. Kubernetes ingress resources are used to configure the ingress rules and routes for individual Kubernetes services. Using an ingress controller and ingress rules, a single IP address can be used to route traffic to multiple services in a Kubernetes cluster.

This example shows you how to deploy the HAProxy ingress controller in an Azure Kubernetes Service cluster. The cert-manager project is used to automatically generate and configure Let's Encrypt certificates. Finally, an application is run in the cluster, accessible over a single host-name (FQDN).


## Prerequisites
* Install [Cert-Manager](https://github.com/amy88ma/Ingress-Configuration/blob/a17f7a435772cd1acf72840a83b27d4d4bc14be9/Jupyter%20Notebooks/Install_cert-manager.ipynb).

Install cert-manager to do the work with Kubernetes to request a certificate and respond to the challenge to validate it. You can use Helm or plain Kubernetes manifest to install cert-manager.

* Install Helm.

Helm is used to deploy the HPCC chart, and create the ingress-nginx controller in this example.

* Deploy the HAProxy controller.

A kubernetes ingress controller is designed to be the access point for HTTP and HTTPS traffic to the software running within your cluster. The controller does this by providing an HTTP proxy service supported by your cloud provider’s load balancer.

* Have the HPCC helm chart deployed.

## Add a record to DNS zone
1) To add a record, use the following command to get the External-IP of the ingress-HAProxy controller, which was deployed under the namespace 'ingress-controller'.  Configure an FQDN-- Fully Qualified Domain Name -- for the ingress controller IP address. In the following example, the IP address is as follows:

In [ ]:
$ kubectl get svc -n ingress-controller

NAME              TYPE           CLUSTER-IP     EXTERNAL-IP    PORT(S)                      AGE
haproxy-ingress   LoadBalancer   10.0.241.105   20.72.71.112   80:31252/TCP,443:31722/TCP   24s

In [ ]:
IP="20.72.71.112"

2) Then, create a DNS name, to associate with the public IP address. This example creates a DNS name called 'hpcc-ingress':

The external IP that is allocated to the ingress-controller is the IP to which all incoming traffic should be routed. To enable this, add it to a DNS zone you control.

In [ ]:
DNSNAME="hpcc-ingress"

3) Get the resource-id of the public IP, with the following command:

In [ ]:
PUBLICIPID=$(az network public-ip list --query "[?ipAddress!=null]|[?contains(ipAddress, '20.72.71.112')].[id]" --output tsv)

4) Next, update your public IP address with the previously created DNS name, 'hpcc-ingress':

The output is as follows:

In [ ]:
az network public-ip update --ids $PUBLICIPID --dns-name $DNSNAME

In [ ]:
{
  "ddosSettings": null,
  "deleteOption": null,
  "dnsSettings": {
    "domainNameLabel": "haproxy-hpcc",
    "fqdn": "haproxy-hpcc.eastus2.cloudapp.azure.com",
    "reverseFqdn": null
  },
  "etag": "W/\"9cb49bba-edda-4d2d-bd03-a72369975f0d\"",
  "extendedLocation": null,
  "id": "/subscriptions/49219efc-701f-4c7e-a2ac-c600308a69e3/resourceGroups/mc_aks-1nodepool-sa-amy_aks-1nodepool-sa-amy-eastus2_eastus2/providers/Microsoft.Network/publicIPAddresses/kubernetes-a30166516976d452aa28de907c896961",
  "idleTimeoutInMinutes": 4,
  "ipAddress": "20.85.19.234",
  "ipConfiguration": {
    "etag": null,
    "id": "/subscriptions/49219efc-701f-4c7e-a2ac-c600308a69e3/resourceGroups/mc_aks-1nodepool-sa-amy_aks-1nodepool-sa-amy-eastus2_eastus2/providers/Microsoft.Network/loadBalancers/kubernetes/frontendIPConfigurations/a30166516976d452aa28de907c896961",
    "name": null,
    "privateIpAddress": null,
    "privateIpAllocationMethod": null,
    "provisioningState": null,
    "publicIpAddress": null,
    "resourceGroup": "mc_aks-1nodepool-sa-amy_aks-1nodepool-sa-amy-eastus2_eastus2",
    "subnet": null
  },
  "ipTags": [],
  "linkedPublicIpAddress": null,
  "location": "eastus2",
  "migrationPhase": null,
  "name": "kubernetes-a30166516976d452aa28de907c896961",
  "natGateway": null,
  "provisioningState": "Succeeded",
  "publicIpAddressVersion": "IPv4",
  "publicIpAllocationMethod": "Static",
  "publicIpPrefix": null,
  "resourceGroup": "mc_aks-1nodepool-sa-amy_aks-1nodepool-sa-amy-eastus2_eastus2",
  "resourceGuid": "09c64a14-e168-45f0-9f0f-6cf35393a1e6",
  "servicePublicIpAddress": null,
  "sku": {
    "name": "Standard",
    "tier": "Regional"
  },
  "tags": {
    "kubernetes-cluster-name": "kubernetes",
    "service": "ingress-controller/haproxy-ingress"
  },
  "type": "Microsoft.Network/publicIPAddresses",
  "zones": [
    "1",
    "2",
    "3"
  ]
}

5) To display the FQDN needed for later to add to the ingress route host name, use the following command:

You should see something like this:

In [ ]:
$ az network public-ip show --ids $PUBLICIPID --query "[dnsSettings.fqdn]" --output tsv
hpcc-ingress.eastus2.cloudapp.azure.com

## Create a CA cluster issuer

In order to begin issuing certificates, you will need to set up a ClusterIssuer. ClusterIssuer works across all namespaces. Create a cluster issuer, such as cluster-issuer.yaml, using the following example manifest. Update the email address with a valid address from your organization.

This email required by Let’s Encrypt and used to notify you of certificate expiration and updates. 

Note:

If cert-manager is not installed, an error will be given, similar to:
```
error: unable to recognize "cluster-issuer.yaml": no matches for kind "ClusterIssuer" in version "cert-manager.io/v1"
```

In [ ]:
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: letsencrypt
spec:
  acme:
    server: https://acme-v02.api.letsencrypt.org/directory
    email: example@email.com # Replace with a valid email
    privateKeySecretRef:
      name: letsencrypt
    solvers:
    - http01:
        ingress:
          class: nginx
          podTemplate:
            spec:
              nodeSelector:
                "kubernetes.io/os": linux

Create the issuer:

In [ ]:
$ kubectl apply -f cluster-issuer.yaml
clusterissuer.cert-manager.io/letsencrypt created

## Create Ingress route
An ingress controller and a certificate management solution have been configured, applications can be run. Create a file with the name "hpcc-ingress.yaml"

In [ ]:
apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  name: hpcc-ingress
  annotations:
    kubernetes.io/ingress.class: haproxy
    nginx.ingress.kubernetes.io/use-regex: "true"
    nginx.ingress.kubernetes.io/rewrite-target: /$1
spec:
  rules:
   - http:
      paths:
      - backend:
          serviceName: eclwatch
          servicePort: 8010
        path: /eclwatch(/|$)(.*)
      - backend:
          serviceName: eclqueries
          servicePort: 8002
        path: /eclqueries(/|$)(.*)
      - backend:
          serviceName: esdl-sandbox
          servicePort: 8899
        path: /esdl(/|$)(.*)
      - backend:
          serviceName: sql2ecl
          servicePort: 8510
        path: /wssql(/|$)(.*)
      - backend:
          serviceName: eclservices
          servicePort: 8010
        path: /(.*)

The path names correspond to the respective port numbers. To test the ingress later, the path names will follow the FQDN:

hpcc-ingress.eastus2.cloudapp.azure.com/eclwatch

Apply the Ingress route:

In [ ]:
$ kubectl apply -f hpcc-ingress.yaml
ingress.networking.k8s.io/hpcc-ingress created

## Upgrade the Ingress route
The ingress route created previously is now running on your kubernetes cluster. To make the service publicly available, update the Kubernetes ingress resource. The ingress resource configures the rules that route traffic to one of the applications.

Refer to a previous step where FDQN was made for the ingress controller IP address. It will be the host in the yaml file shown below. The DNS name you specified is used in the FQDN. example: DNS_NAME.eastus.cloudapp.azure.com In this case, it is: hpcc-ingress.eastus2.cloudapp.azure.com

Display your FQDN needed for the host name, using the following command:

In [ ]:
$ az network public-ip show --ids $PUBLICIPID --query "[dnsSettings.fqdn]" --output tsv
hpcc-ingress.eastus2.cloudapp.azure.com

Open the file for the Ingress route, named "hpcc-ingres.yaml"

In [ ]:
apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  name: hpcc-ingress
  annotations:
    kubernetes.io/ingress.class: haproxy
    nginx.ingress.kubernetes.io/use-regex: "true"
    nginx.ingress.kubernetes.io/rewrite-target: /$1
    cert-manager.io/cluster-issuer: letsencrypt
spec:
  tls:
  - hosts:
    - hpcc-ingress.eastus2.cloudapp.azure.com
    secretName: tls-secret
  rules:
  - host: hpcc-ingress.eastus2.cloudapp.azure.com
    http:
      paths:
      - backend:
          serviceName: eclwatch
          servicePort: 8010
        path: /eclwatch(/|$)(.*)
      - backend:
          serviceName: eclqueries
          servicePort: 8002
        path: /eclqueries(/|$)(.*)
      - backend:
          serviceName: esdl-sandbox
          servicePort: 8899
        path: /esdl(/|$)(.*)
      - backend:
          serviceName: sql2ecl
          servicePort: 8510
        path: /wssql(/|$)(.*)
      - backend:
          serviceName: eclservices
          servicePort: 8010
        path: /(.*)

Apply the updated ingress route:

In [ ]:
kubectl apply -f hpcc-ingress-route.yaml

The output should look like:

In [ ]:
ingress.networking.k8s.io/hpcc-ingress configured

## Verify that an object has been created¶
Next, a certificate resource must be created. The certificate resource defines the desired X.509 certificate. Cert-manager has automatically created a certificate object for you using ingress-shim, which is automatically deployed with cert-manager since v0.2.2.

Cert-manager will read the annotations configured in the above YAML file, and use them to create a certificate, which you can request and see:

To verify that the certificate was created successfully, use the command below. READY must be True.

In [2]:
kubectl get certificates

No resources found in default namespace.


## Test the Ingress configuration

Open a web browser to the FQDN created previously. The FQDN name acts as a host for the ingress External IP address. Notice you are redirect to use HTTPS, the connection is secure, and the certificate is trusted.

Open web browser, and visit the FQDN name followed by 'eclwatch' path name defined in the previous Ingress file, to visit the backend eclwatch, through the port number 8010.